## 1. fashion MNIST

In [137]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.transforms import ToTensor

train_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [153]:
import torch.nn as nn
# 위와 같은 모델을 클래스를 이용하여 작성
class net(nn.Module): # nn.Module 상속(inheritance)
    def __init__(self): # 초기값
        super(net, self).__init__() # nn.Module의 함수를 model에서 사용 
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64,10)

    def forward(self, x):
        #x = self.flatten(x,start_dim=1)/255 # flatten
        x = x.view(-1, 28*28).float()
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        x = nn.functional.softmax(x,dim=1)
        return x

In [154]:
model = net()

# loss and optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [155]:
# mini batch
batch_size = 100
train_loader = torch.utils.data.DataLoader(
    dataset = train_data,
    batch_size = batch_size,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    dataset = test_data,
    batch_size = batch_size,
    shuffle = False
)

In [156]:
def train(model, criterion, optimizer, data):
    model.train() # training
    for X,y in data:
        y_pred = model(X)
        loss = criterion(y_pred,y)
        
        optimizer.zero_grad() # zero_grad(): 미분값 초기화
        loss.backward() # backpropagation
        optimizer.step() # update weights and biases

In [157]:
def evaluation(model, criterion, optimizer, data):
    model.eval() #evaluation
    test_loss = 0
    accuracy = 0

    with torch.no_grad(): # no update
        for X, y in data:
            y_pred = model(X)
            test_loss += criterion(y_pred, y)
            
            prediction = y_pred.max(1)[1] # max indices
            corrects = (prediction == y)
            accuracy += corrects.sum().float() / float( y.size(0) )
    return test_loss, accuracy      

In [161]:
nums_epoch = 10
for epoch in range(nums_epoch):
    train(model, criterion, optimizer, train_loader)
    
    if(epoch%2==0):
        E, accu = evaluation(model, criterion, optimizer, test_loader)
        print(epoch, "loss=", E.item(), "accu=", accu.item())

0 loss= 158.18392944335938 accu= 87.80998229980469
2 loss= 159.3319549560547 accu= 86.66999053955078
4 loss= 158.4934539794922 accu= 87.6500015258789
6 loss= 158.17405700683594 accu= 87.8899917602539
8 loss= 157.72488403320312 accu= 88.31998443603516


## 2. 아래의 글을 읽고 500자 이내로 감상문을 작성하세요.

In [ ]:
이 글에서 계속 주장하는 것은 고정관념을 깨야 한다. 이렇게 말하고 있는데 필자는 다르게 생각한다. 
우선 필자의 생각을 말하기 앞서 베이커 그룹과 알파폴드의 문제 접근 차이점이다. 
베이커 그룹은 다른 기술을 만드는 것에 초점을 두었다면 알파폴드는 기존의 기술을 찾는 쪽이었다.  
비록 알파폴드가 승리했지만 이것이 알파폴드가 모든 면에서 이겼다고 보기는 어렵다.     
오히려 새로운 기술을 발견한 베이커 그룹이 향후 ai 기술을 더 발전시킬지도 모르는 일이다. 
이것을 비유적으로 표현하면 연구는 검은 방을 밝히는 것이라면 ai는 밝은 방을 걷는 것이다.  
즉 ai는 현재로서는 기술을 만드는 기술은 아니다. 새로운 기술을 찾는 것은 아직 인간의 영역이다. 
그렇기에 베이커 그룹은 실패했다고 볼 수 없다. 
결국 이 글에서 함정은 고정관념을 깨야 한다고 말하면서 어떻게 깨야 하는가를 말하고 있지 않다.
단지 결과만 가지고 베이커 그룹의 노력이 물거품인양 보이게 만들 수 있는 것이다.
